In [1]:
import os
import json

import torch as T
import torch.nn as nn
from torch import optim
import numpy as np
import pandas as pd

import torch_utils
from torch import distributions

import gymnasium as gym
import gymnasium_robotics as gym_robo
import models
import cnn_models
import rl_agents
import rl_callbacks
import helper
import gym_helper
import wandb_support
import wandb
import gym_helper
import ale_py

# from mpi4py import MPI

In [ ]:
import mpi4py

In [ ]:
gym_robo.__version__

In [ ]:
torch.cuda.empty_cache()
print(f"CUDA version: {torch.version.cuda}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of CUDA devices: {torch.cuda.device_count()}")
print(torch.cuda.get_device_name(0))
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__CUDA Device Name:',torch.cuda.get_device_name(0))
print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
print('Memory Usage:')
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

# TEST

In [ ]:
gym_robo.register_robotics_envs()

In [ ]:
gym.envs.registration.registry

In [ ]:
wandb.login(key='758ac5ba01e12a3df504d2db2fec8ba4f391f7e6')

In [ ]:
env = gym.make('FetchPush-v2', max_episode_steps=100, render_mode='rgb_array')
env = gym.wrappers.RecordVideo(env, 'test/', episode_trigger=lambda i: i%1==0)

episodes = 10


for episode in range(episodes):
    done = False
    obs, _ = env.reset()
    while not done:
        obs, r, term, trunc, dict = env.step(env.action_space.sample())
        if term or trunc:
            done = True
env.close()

In [ ]:
env = gym.make("FetchReach-v2")
env.reset()
obs, reward, terminated, truncated, info = env.step(env.action_space.sample())

# The following always has to hold:
assert reward == env.compute_reward(obs["achieved_goal"], obs["desired_goal"], info)
assert truncated == env.compute_truncated(obs["achieved_goal"], obs["desired_goal"], info)
assert terminated == env.compute_terminated(obs["achieved_goal"], obs["desired_goal"], info)

In [ ]:
env.compute_reward()

In [ ]:
env = gym.make('FetchPush-v2', render_mode='rgb_array')

In [ ]:
if hasattr(env, "distance_threshold"):
    print('true')
else:
    print('false')

In [ ]:
if env.get_wrapper_attr("distance_threshold"):
    print('true')

In [ ]:
print(dir(env))


# DDPG

In [ ]:
env = gym.make('LunarLanderContinuous-v2')

In [ ]:
device='cuda'

In [ ]:
# # build actor

# dense_layers = [
#     (
#         400,
#         "relu",
#         {
#             "variance scaling": {
#                 "scale": 1.0,
#                 "mode": "fan_in",
#                 "distribution": "uniform",
#             }
#         },
#     ),
#     (
#         300,
#         "relu",
#         {
#             "variance scaling": {
#                 "scale": 1.0,
#                 "mode": "fan_in",
#                 "distribution": "uniform",
#             }
#         },
#     )
# ]

# actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, optimizer='Adam',
#                           optimizer_params={'weight_decay':0.01}, learning_rate=0.001, normalize_layers=True)

In [ ]:
# build actor

dense_layers = [
    (
        400,
        "relu",
        {
            "default": {
                
            }
        },
    ),
    (
        300,
        "relu",
        {
            "default": {
                
            }
        },
    )
]

# output_kernel = {"uniform":{"a":-0.003, "b":0.003}}
output_kernel = {"default":{}}

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, output_layer_kernel=output_kernel,
                          optimizer='Adam', optimizer_params={'weight_decay':0.0}, learning_rate=0.001,
                          normalize_layers=False, device=device)

In [ ]:
actor

In [ ]:
# # build critic

# state_layers = [
#     (
#         400,
#         "relu",
#         {
#             "variance scaling": {
#                 "scale": 1.0,
#                 "mode": "fan_in",
#                 "distribution": "uniform",
#             }
#         },
#     )
# ]

# merged_layers = [
#     (
#         300,
#         "relu",
#         {
#             "variance scaling": {
#                 "scale": 1.0,
#                 "mode": "fan_in",
#                 "distribution": "uniform",
#             }
#         },
#     )
# ]


# critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers,
#                             optimizer='Adam', optimizer_params={'weight_decay':0.01}, learning_rate=0.002, normalize_layers=True)

In [ ]:
# build critic
# build actor

state_layers = []

merged_layers = [
    (
        400,
        "relu",
        {
            "default": {
                
            }
        },
    ),
    (
        300,
        "relu",
        {
            "default": {
                
            }
        },
    )
]

# output_kernel = {"uniform":{"a":-0.003, "b":0.003}}
output_kernel = {"default":{}}

critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers,
                            output_layer_kernel=output_kernel, optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},learning_rate=0.001, normalize_layers=False,
                            device=device)

In [ ]:
critic

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 100000, device=device)
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.1, device=device)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.99,
                            tau=0.005,
                            replay_buffer=replay_buffer,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('LunarLander-v2-continuous')],
                            device=device)

In [ ]:
ddpg_agent.critic_model

In [ ]:
ddpg_agent.target_critic_model

In [ ]:
ddpg_agent.train(500)

In [ ]:
ddpg_agent.test(10, True, 1)

# Actor Critic

In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
dense_layers = [
    (128, 'relu', "kaiming normal"),
    (256, 'relu', "kaiming normal"),
    ]



In [ ]:
policy_model = models.PolicyModel(env=env, dense_layers=dense_layers, optimizer='Adam', learning_rate=0.001,)

In [ ]:
for param in policy_model.parameters():
    print(param)

In [ ]:
value_model = models.ValueModel(env, dense_layers=dense_layers, optimizer='Adam', learning_rate=0.001)

In [ ]:
value_model

In [ ]:
for params in value_model.parameters():
    print(params)

In [ ]:
actor_critic = rl_agents.ActorCritic(env,
                                     policy_model,
                                     value_model,
                                     discount=0.99,
                                     policy_trace_decay=0.5,
                                     value_trace_decay=0.5,
                                     callbacks=[rl_callbacks.WandbCallback('CartPole-v1-Actor-Critic')])

In [ ]:
actor_critic.train(200)

In [ ]:
actor_critic.test(10, True, 1)

# REINFORCE

In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
dense_layers = [
    (128, 'relu', {
                    "kaiming normal": {
                        "a":1.0,
                        "mode":'fan_in'
                    }
                },
    ),
    # (256, 'relu', {
    #                 "kaiming_normal": {
    #                     "a":0.0,
    #                     "mode":'fan_in'
    #                 }
    #             },
    # )
    ]

In [ ]:
dense_layers = [(128, 'relu', "kaiming normal")]

In [ ]:
value_model = models.ValueModel(env, dense_layers, 'Adam', 0.001)

In [ ]:
for param in value_model.parameters():
    print(param)

In [ ]:
policy_model = models.PolicyModel(env, dense_layers, 'Adam', 0.001)

In [ ]:
for param in policy_model.parameters():
    print(param)

In [ ]:
reinforce = rl_agents.Reinforce(env, policy_model, value_model, 0.99, [rl_callbacks.WandbCallback('CartPole-v0_REINFORCE', chkpt_freq=100)])

In [ ]:
reinforce.train(200, True, 50)

In [ ]:
reinforce.test(10, True, 1)

# DDPG w/CNN

In [ ]:
env = gym.make('CarRacing-v2')

In [ ]:
cnn_layers = [
    # {
    #     "batchnorm":
    #     {
    #         "num_features":3
    #     }
    # },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 7,
            "stride": 3,
            "padding": 'valid',
            "bias": False
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 5,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 3,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
]

In [ ]:
cnn = cnn_models.CNN(cnn_layers, env)

In [ ]:
cnn

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
]

actor = models.ActorModel(env, cnn_model=cnn, dense_layers=dense_layers, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.0001, normalize=False)

In [ ]:
actor

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env, cnn_model=cnn, state_layers=state_layers, merged_layers=merged_layers, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.0001, normalize=False)

In [ ]:
critic

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape=(1,))
noise = helper.OUNoise(shape=env.action_space.shape, mean=0.0, theta=0.15, sigma=0.01, dt=1.0, device='cuda')

In [ ]:
ddpg_agent = rl_agents.DDPG(
    env,
    actor,
    critic,
    discount=0.98,
    tau=0.05,
    action_epsilon=0.2,
    replay_buffer=replay_buffer,
    batch_size=128,
    noise=noise,
    callbacks=[rl_callbacks.WandbCallback("CarRacing-v2")]
)

In [ ]:
ddpg_agent.train(1000, True, 10)

In [ ]:
wandb.finish()

In [ ]:
wandb.login()

# HER

In [ ]:
env = gym.make("Reacher-v4")

In [ ]:
_,_ = env.reset()

In [ ]:
achieved_goal = gym_helper.reacher_achieved_goal(env)
action = env.action_space.sample()
env.step(action)
print(f'observation: {env.get_wrapper_attr("_get_obs")()}')
print(f'distance to goal: {env.get_wrapper_attr("_get_obs")()[8::]}')
print(f'fingertip: {env.get_wrapper_attr("get_body_com")("fingertip")}')
print(f'target: {env.get_wrapper_attr("get_body_com")("target")}')

In [ ]:
next_achieved_goal = env.get_wrapper_attr("_get_obs")()[8::]
desired_goal = [0.0, 0.0, 0.0]

In [ ]:
reward_func(env, action, achieved_goal, next_achieved_goal, desired_goal, 0.05)

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
desired_goal_func(env).shape

In [ ]:
# build actor

dense_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
]

actor = models.ActorModel(env,
                          cnn_model=None,
                          dense_layers=dense_layers,
                          goal_shape=(3,),
                          optimizer="Adam",
                          optimizer_params={'weight_decay':0.0},
                          learning_rate=0.0001, normalize=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env,
                            cnn_model=None,
                            state_layers=state_layers,
                            merged_layers=merged_layers,
                            goal_shape=(3,),
                            optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},
                            learning_rate=0.0001,
                            normalize=False)

In [ ]:
goal_shape = desired_goal_func(env).shape
replay_buffer = helper.ReplayBuffer(env, 100000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape,
#                        mean=0.0,
#                        theta=0.05,
#                        sigma=0.15,
#                        dt=1.0, device='cuda')

noise=helper.NormalNoise(shape=env.action_space.shape,
                         mean = 0.0,
                         stddev=0.05,
                         )

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.2,
                            replay_buffer=replay_buffer,
                            batch_size=256,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('Reacher-v4')])

In [ ]:
her = rl_agents.HER(ddpg_agent,
                    strategy='future',
                    num_goals=4,
                    tolerance=0.001,
                    desired_goal=desired_goal_func,
                    achieved_goal=achieved_goal_func,
                    reward_fn=reward_func)

In [ ]:
her.train(10, 50, 16, 40, True, 1000)

In [ ]:
wandb.finish()

In [ ]:
her.test(10, True, 1)

In [ ]:
her.save()

In [ ]:
her.agent.goal_normalizer.running_std

In [ ]:
loaded_her = rl_agents.HER.load("/workspaces/RL_Agents/pytorch/src/app/assets/models/her")

In [ ]:
loaded_her.agent.replay_buffer.sample(10)

In [ ]:
loaded_her.agent.state_normalizer.running_cnt

In [ ]:
loaded_her.get_config()

In [ ]:
loaded_her.test(10, True, 1)

In [ ]:
10e4

# HER w/CNN

In [ ]:
env = gym.make('CarRacing-v2')

In [ ]:
_,_ = env.reset()

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
desired_goal(env).shape

In [ ]:
cnn_layers = [
    # {
    #     "batchnorm":
    #     {
    #         "num_features":3
    #     }
    # },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 7,
            "stride": 3,
            "padding": 'valid',
            "bias": False
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 5,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 3,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
]

cnn = cnn_models.CNN(cnn_layers, env)

In [ ]:
# build actor

dense_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
]

actor = models.ActorModel(env,
                          cnn_model=cnn,
                          dense_layers=dense_layers,
                          goal_shape=(1,),
                          optimizer="Adam",
                          optimizer_params={'weight_decay':0.0},
                          learning_rate=0.001, normalize=False)

In [ ]:
actor

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env,
                            cnn_model=cnn,
                            state_layers=state_layers,
                            merged_layers=merged_layers,
                            goal_shape=(1,),
                            optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},
                            learning_rate=0.001,
                            normalize=False)

In [ ]:
critic

In [ ]:
goal_shape = desired_goal_func(env).shape
replay_buffer = helper.ReplayBuffer(env, 100000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape,
#                        mean=0.0,
#                        theta=0.05,
#                        sigma=0.15,
#                        dt=1.0, device='cuda')

noise=helper.NormalNoise(shape=env.action_space.shape,
                         mean = 0.0,
                         stddev=0.05,
                         )

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.2,
                            replay_buffer=replay_buffer,
                            batch_size=256,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('CarRacing-v2')])

In [ ]:
ddpg_agent.actor_model

In [ ]:
her = rl_agents.HER(ddpg_agent,
                    strategy='future',
                    num_goals=4,
                    tolerance=1,
                    desired_goal=desired_goal_func,
                    achieved_goal=achieved_goal_func,
                    reward_fn=reward_func)

In [ ]:
her.agent.actor_model

In [ ]:
her.train(num_epochs=20,
          num_cycles=50,
          num_episodes=16,
          num_updates=40,
          render=True,
          render_freq=20
        )

In [ ]:
her = rl_agents.HER.load("/workspaces/RL_Agents/pytorch/src/app/models/her")

In [ ]:
wandb.finish()

In [ ]:
# reset environment
state, _ = her.agent.env.reset()
# instantiate empty lists to store current episode trajectory
states, actions, next_states, dones, state_achieved_goals, \
next_state_achieved_goals, desired_goals = [], [], [], [], [], [], []
# set desired goal
desired_goal = her.desired_goal_func(her.agent.env)
# set achieved goal
state_achieved_goal = her.achieved_goal_func(her.agent.env)
# add initial state and goals to local normalizer stats
her.state_normalizer.update_local_stats(state)
her.goal_normalizer.update_local_stats(desired_goal)
her.goal_normalizer.update_local_stats(state_achieved_goal)
# set done flag
done = False
# reset episode reward to 0
episode_reward = 0
# reset steps counter for the episode
episode_steps = 0

while not done:
    # get normalized values for state and desired goal
    state_norm = her.state_normalizer.normalize(state)
    desired_goal_norm = her.goal_normalizer.normalize(desired_goal)
    # get action
    action = her.agent.get_action(state_norm, desired_goal_norm, grad=False)
    # take action
    next_state, reward, term, trunc, _ = her.agent.env.step(action)
    # get next state achieved goal
    next_state_achieved_goal = her.achieved_goal_func(her.agent.env)
    # add next state and next state achieved goal to normalizers
    her.state_normalizer.update_local_stats(next_state)
    her.goal_normalizer.update_local_stats(next_state_achieved_goal)
    # store trajectory in replay buffer (non normalized!)
    her.agent.replay_buffer.add(state, action, reward, next_state, done,\
                                    state_achieved_goal, next_state_achieved_goal, desired_goal)
    
    # append step state, action, next state, and goals to respective lists
    states.append(state)
    actions.append(action)
    next_states.append(next_state)
    dones.append(done)
    state_achieved_goals.append(state_achieved_goal)
    next_state_achieved_goals.append(next_state_achieved_goal)
    desired_goals.append(desired_goal)

    # add to episode reward and increment steps counter
    episode_reward += reward
    episode_steps += 1
    # update state and state achieved goal
    state = next_state
    state_achieved_goal = next_state_achieved_goal
    # update done flag
    if term or trunc:
        done = True

In [ ]:
# package episode states, actions, next states, and goals into trajectory tuple
trajectory = (states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals)

In [ ]:
states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals = trajectory

In [ ]:
for idx, (s, a, ns, d, sag, nsag, dg) in enumerate(zip(states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals)):
    print(f'a={a}, d={d}, sag={sag}, nsag={nsag}, dg={dg}')

In [ ]:
strategy = "future"
num_goals = 4

# loop over each step in the trajectory to set new achieved goals, calculate new reward, and save to replay buffer
for idx, (state, action, next_state, done, state_achieved_goal, next_state_achieved_goal, desired_goal) in enumerate(zip(states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals)):

    if strategy == "final":
        new_desired_goal = next_state_achieved_goals[-1]
        new_reward = her.reward_fn(state_achieved_goal, next_state_achieved_goal, new_desired_goal)
        print(f'transition: action={action}, reward={new_reward}, done={done}, state_achieved_goal={state_achieved_goal}, next_state_achieved_goal={next_state_achieved_goal}, desired_goal={new_desired_goal}')
        her.agent.replay_buffer.add(state, action, new_reward, next_state, done, state_achieved_goal, next_state_achieved_goal, new_desired_goal)

    if strategy == 'future':
        for i in range(num_goals):
            if idx + i + 1 >= len(states):
                break
            goal_idx = np.random.randint(idx + 1, len(states))
            new_desired_goal = next_state_achieved_goals[goal_idx]
            new_reward = her.reward_fn(state_achieved_goal, next_state_achieved_goal, new_desired_goal)
            print(f'transition: action={action}, reward={new_reward}, done={done}, state_achieved_goal={state_achieved_goal}, next_state_achieved_goal={next_state_achieved_goal}, desired_goal={new_desired_goal}')
            her.agent.replay_buffer.add(state, action, new_reward, next_state, done, state_achieved_goal, next_state_achieved_goal, new_desired_goal)
    

    


In [ ]:
s, a, r, ns, d, sag, nsag, dg = her.agent.replay_buffer.sample(100)

In [ ]:
for i in range(100):
    print(f'{i}: a={a[i]}, r={r[i]}, d={d[i]}, sag={sag[i]}, nsag={nsag[i]}, dg={dg[i]} ')

# HER Pendulum

In [ ]:
env = gym.make('Pendulum-v1')

In [ ]:
# build actor

dense_layers = [
    (
        400,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        300,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, optimizer='Adam',
                          optimizer_params={'weight_decay':0.01}, learning_rate=0.001, normalize=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.001, normalize=False)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 100000, (3,))
noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.99,
                            tau=0.005,
                            replay_buffer=replay_buffer,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('Pendulum-v1')])

In [ ]:
def desired_goal_func(env):
    return np.array([0.0, 0.0, 0.0])

def achieved_goal_func(env):
    return env.get_wrapper_attr('_get_obs')()

def reward_func(env):
    pass

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='none',
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=10.0
)

In [ ]:
her.agent.critic_model

In [ ]:
her.agent.target_critic_model

In [ ]:
her.train(1,1,100,1)

In [ ]:
wandb.finish()

In [ ]:
state = env.observation_space.sample()
state

In [ ]:
her.agent.state_normalizer.normalize(state)

In [ ]:
goal = her.desired_goal_func(her.agent.env)
goal

In [ ]:
her.agent.goal_normalizer.normalize(goal)

In [ ]:
def remove_renders(folder_path):
    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file has a .mp4 or .meta.json extension
        if filename.endswith(".mp4") or filename.endswith(".meta.json"):
            # Construct the full file path
            file_path = os.path.join(folder_path, filename)
            # Remove the file
            os.remove(file_path)

In [ ]:
remove_renders("/workspaces/RL_Agents/pytorch/src/app/assets/models/ddpg/renders/training")

# HER Fetch-Reach (Robotics)

In [ ]:
env = gym.make("FetchReach-v2", max_episode_steps=50)

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
achieved_goal_func(env)

In [ ]:
env.get_wrapper_attr("_get_obs")()

In [ ]:
# reset env state
env.reset()

In [ ]:
goal_shape = desired_goal_func(env).shape

In [ ]:
goal_shape

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, goal_shape=goal_shape, optimizer='Adam',
                          optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
actor

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
               
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, goal_shape=goal_shape, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
critic

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.05)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.2,
                            replay_buffer=replay_buffer,
                            batch_size=256,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback("FetchReach-v2")])

In [ ]:
ddpg_agent.critic_model

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='future',
    tolerance=0.05,
    num_goals=4,
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=5.0
)

In [ ]:
her.train(num_epochs=50,
          num_cycles=50,
          num_episodes=16,
          num_updates=40,
          render=True,
          render_freq=1000)

In [ ]:
states, action, rewards, next_states, dones, achieved_goals, next_achieved_goals, desired_goals = her.agent.replay_buffer.sample(2)

In [ ]:
desired_goals

In [ ]:
her.agent.env.get_wrapper_attr("distance_threshold")

In [ ]:
# get success
her.agent.env.get_wrapper_attr("_is_success")(achieved_goal_func(her.agent.env), desired_goal_func(her.agent.env))

In [ ]:
her.agent.env.get_wrapper_attr("goal_distance")(next_state_achieved_goal, desired_goal, None)

In [ ]:
pusher_her = rl_agents.HER.load("/workspaces/RL_Agents/pytorch/src/app/assets/models/her")

In [ ]:
pusher_her.agent.env.reset()

In [ ]:
pusher_her.get_config()

In [ ]:
wandb.finish()

In [ ]:
np.linalg.norm(pusher_her.agent.env.get_wrapper_attr("get_body_com")("goal") - pusher_her.agent.env.get_wrapper_attr("get_body_com")("object"))

In [ ]:
pusher_her.agent.replay_buffer.get_config()

In [ ]:

pusher_her.agent.replay_buffer.desired_goals

In [ ]:
## TEST ENV
env = gym.make("Pusher-v5", render_mode="rgb_array")

In [ ]:
env = gym.wrappers.RecordVideo(
                    env,
                    "/renders/training",
                    episode_trigger=lambda x: True,
                )


In [ ]:
state, _ = env.reset()

for i in range(1000):
# take action
    next_state, reward, term, trunc, _ = env.step(env.action_space.sample())
env.close()

# HER Fetch Push (Robitics)

In [ ]:
env = gym.make('FetchPush-v2')

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
# reset env state
env.reset()

In [ ]:
goal_shape = desired_goal_func(env).shape

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, goal_shape=goal_shape, optimizer='Adam',
                          optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
               
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, goal_shape=goal_shape, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.05)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.3,
                            replay_buffer=replay_buffer,
                            batch_size=128,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback("FetchPush-v2")],
                            save_dir="fetch_push/models/ddpg/"
                            )

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='final',
    tolerance=0.05,
    num_goals=4,
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=5.0,
    save_dir="fetch_push/models/her/"
)

In [ ]:
her.train(num_epochs=50,
          num_cycles=50,
          num_episodes=16,
          num_updates=40,
          render=True,
          render_freq=1000)

# TESTING MULTITHREADING

In [ ]:
env = gym.make('FetchPush-v2')

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
# reset env state
env.reset()

In [ ]:
goal_shape = desired_goal_func(env).shape

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, goal_shape=goal_shape, optimizer='Adam',
                          optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
               
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, goal_shape=goal_shape, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.05)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.3,
                            replay_buffer=replay_buffer,
                            batch_size=128,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback("FetchPush-v2")],
                            save_dir="fetch_push/models/ddpg/"
                            )

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='final',
    num_workers=4,
    tolerance=0.05,
    num_goals=4,
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=5.0,
    save_dir="fetch_push/models/her/"
)

In [ ]:
her.train()

# TESTING

In [ ]:
# load config
config_path = "/workspaces/RL_Agents/pytorch/src/app/HER_Test/her/config.json"
with open(config_path, 'r') as file:
    config = json.load(file)

In [ ]:
config

In [ ]:
agent = rl_agents.HER.load(config)

In [ ]:
for callback in agent.agent.callbacks:
    print(callback._sweep)

# Co Occurence

In [ ]:
import subprocess

In [ ]:
# Define the path to your JSON configuration file
config_file_path = 'assets/wandb_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    wandb_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(wandb_config)

In [ ]:
# Define the path to your JSON configuration file
config_file_path = 'assets/sweep_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    sweep_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(sweep_config)

In [ ]:
# Save the updated configuration to a train config file
os.makedirs('sweep', exist_ok=True)
train_config_path = os.path.join(os.getcwd(), 'sweep/train_config.json')
with open(train_config_path, 'w') as f:
    json.dump(sweep_config, f)

# Save and Set the sweep config path
sweep_config_path = os.path.join(os.getcwd(), 'sweep/sweep_config.json')
with open(sweep_config_path, 'w') as f:
    json.dump(wandb_config, f)

In [ ]:
command = ['python', 'sweep.py']

# Set the environment variable
os.environ['WANDB_DISABLE_SERVICE'] = 'true'

subprocess.Popen(command)

In [ ]:
# Set the environment variable
os.environ['WANDB_DISABLE_SERVICE'] = 'true'

In [ ]:
# Define the path to your JSON configuration file
config_file_path = 'sweep/sweep_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    sweep_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(sweep_config)

In [ ]:
# Define the path to your JSON configuration file
config_file_path = 'sweep/train_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    train_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(train_config)

In [ ]:
from dash_callbacks import run_agent

sweep_id = wandb.sweep(sweep=sweep_config, project=sweep_config["project"])
# loop over num wandb agents
num_agents = 2
for agent in range(num_agents):
    
)

In [ ]:
sweep_config

In [ ]:
env = gym.make("FetchReach-v2")

In [ ]:
type(env)

In [ ]:
env_spec = env.spec.to_json()

In [ ]:
env_spec

In [ ]:
type(env_spec)

In [ ]:
env = gym.make(gym.envs.registration.EnvSpec.from_json(env_spec))

In [ ]:
env.spec

In [ ]:
def load_env_spec(env_spec_dict):
    # Create a new EnvSpec instance using the dictionary
    env_spec = gym.envs.registration.EnvSpec(**env_spec_dict)
    return env_spec

In [ ]:
load_env_spec(env_spec)

In [ ]:
config_path = 'sweep/agent_config_58.json'

with open(config_path, 'r') as file:
    agent_config = json.load(file)

her = rl_agents.HER.load(agent_config)

In [ ]:
her.get_config()

# TD3

In [ ]:
# env = gym.make('LunarLanderContinuous-v3')
# env = gym.make("BipedalWalker-v3")
env = gym.make("Pendulum-v1")

In [ ]:
env = gym.make(env.spec, render_mode="rgb_array")
save_dir = "/workspaces/RL_Agents/pytorch/src/app/td3_test/td3"
os.makedirs(save_dir + "/renders/training", exist_ok=True)
env = gym.wrappers.RecordVideo(
    env,
    save_dir + "/renders/training",
    episode_trigger=lambda episode_id: (episode_id+1) % render_freq == 0,
)

In [ ]:
env.spec

In [ ]:
env.spec.to_json()

In [ ]:
json_spec

In [ ]:
from gymnasium.envs.registration import EnvSpec

def serialize_env_spec(env_spec):
    """Extracts and serializes the relevant parts of the environment specification."""
    env_spec_dict = {
        "id": env_spec.id,
        "entry_point": env_spec.entry_point,
        "reward_threshold": env_spec.reward_threshold,
        "nondeterministic": env_spec.nondeterministic,
        "max_episode_steps": env_spec.max_episode_steps,
        "order_enforce": env_spec.order_enforce,
        "disable_env_checker": env_spec.disable_env_checker,
        "kwargs": env_spec.kwargs,
        "additional_wrappers": env_spec.additional_wrappers,
        "vector_entry_point": env_spec.vector_entry_point,
    }
    return env_spec_dict

In [ ]:
from_json_spec = gym.envs.registration.EnvSpec.from_json(json_spec)

In [ ]:
from_json_spec

In [ ]:
for key,val in env_spec.items():
    print(f'{key}:{val};{type(val)}')

In [ ]:
gym.envs.registration.EnvSpec.from_json(env_spec)

In [ ]:
env = gym.make(env_spec)

In [ ]:
device='cuda'

In [ ]:
# build actor

dense_layers = [
    (
        256,
        "relu",
        {
            "default": {
                
            }
        },
    ),
    (
        256,
        "relu",
        {
            "default": {
                
            }
        },
    )
]

# output_kernel = {"uniform":{"a":-0.003, "b":0.003}}
output_kernel = {"default":{}}

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, output_layer_kernel=output_kernel,
                          optimizer='Adam', optimizer_params={'weight_decay':0.0}, learning_rate=3e-4,
                          normalize_layers=False, device=device)

In [ ]:
actor

In [ ]:
for param in actor.parameters():
    print(param)

In [ ]:
# build critic

state_layers = []

merged_layers = [
    (
        256,
        "relu",
        {
            "default": {
                
            }
        },
    ),
    (
        256,
        "relu",
        {
            "default": {
                
            }
        },
    )
]

# output_kernel = {"uniform":{"a":-0.003, "b":0.003}}
output_kernel = {"default":{}}

critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers,
                            output_layer_kernel=output_kernel, optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},learning_rate=3e-4, normalize_layers=False,
                            device=device)

In [ ]:
critic

In [ ]:
for params in critic.parameters():
    print(params)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, device=device)
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.1, device=device)

In [ ]:
td3 = rl_agents.TD3(
    env=env,
    actor_model=actor,
    critic_model=critic,
    replay_buffer=replay_buffer,
    batch_size=256,
    noise=noise,
    actor_update_delay=2,
    callbacks=[rl_callbacks.WandbCallback("BipedalWalker-v3")],
    use_mpi=False,
    device=device
)

In [ ]:
for params in td3.critic_model_a.parameters():
    print(params)

In [ ]:
for params in td3.critic_model_b.parameters():
    print(params)

In [ ]:
td3.get_config()

In [ ]:
td3.train(num_episodes=100)

In [ ]:
any(td3.replay_buffer.dones) == 1

In [ ]:
td3.save('src/app/models/td3')

In [ ]:
# load config
with open('/workspaces/RL_Agents/pytorch/src/app/td3_test/td3/config.json', 'r') as file:
    config = json.load(file)
td3 = rl_agents.TD3.load(config)

In [ ]:
for key,val in config.items():
    print(f'{key}:{type(val)};{val}')

In [ ]:
td3.get_config()

In [ ]:
td3.save()

In [ ]:
td3.test(10, True, 1, td3.save_dir)

# Test Replay Buffer

In [2]:
class DummyValueFunction(T.nn.Module):
    def __init__(self, observation_space):
        super(DummyValueFunction, self).__init__()
        self.fc = T.nn.Linear(observation_space.shape[0], 1)
    
    def forward(self, x):
        return self.fc(x)

# Define the replay buffer and the dummy value function
env = gym.make("Pendulum-v1")
buffer = helper.ReplayBuffer(env, buffer_size=1000)
value_function = DummyValueFunction(env.observation_space)

In [3]:
agent = rl_agents.PPO(env, value_function, buffer, timesteps=100)

In [4]:
agent.collect_trajectories()

In [5]:
agent.replay_buffer.counter

200

In [6]:
agent.calculate_advantages_and_returns()

size:200
values shape:torch.Size([200, 1])
values shape:torch.Size([201, 1])
advantages shape:torch.Size([200])
advantages shape:torch.Size([200, 1])
values shape:torch.Size([200, 1])
returns shape:torch.Size([200, 1])


(tensor([[-112.1353],
         [-114.5213],
         [-116.7986],
         [-118.7992],
         [-120.1904],
         [-120.7445],
         [-120.0259],
         [-117.7691],
         [-113.9555],
         [-108.7294],
         [-104.6373],
         [-101.8969],
         [-100.5639],
         [-100.5997],
         [-101.5364],
         [-103.3182],
         [-105.8409],
         [-108.7868],
         [-112.1402],
         [-115.2835],
         [-118.3307],
         [-121.2949],
         [-123.5464],
         [-125.1345],
         [-125.9552],
         [-125.0506],
         [-122.7628],
         [-119.0149],
         [-114.7446],
         [-111.6655],
         [-109.6652],
         [-108.6532],
         [-108.8248],
         [-109.7514],
         [-111.4926],
         [-113.6240],
         [-115.9841],
         [-118.1216],
         [-119.9740],
         [-121.1823],
         [-121.4573],
         [-120.4901],
         [-118.0695],
         [-114.0748],
         [-109.3129],
         [

In [ ]:
# Set the number of timesteps
T_timesteps = 100

# Collect trajectories using random actions
state, _ = env.reset()
done = False
for _ in range(T_timesteps):
    if not done:
        action = env.action_space.sample()  # Random action
        next_state, reward, term, trunc, _ = env.step(action)
        if term or trunc:
            done = True
        buffer.add(state, action, reward, next_state, done)
        state = next_state

# print number of timesteps completed
print(f'num timesteps:{buffer.counter}')

# Calculate and store advantages
values = buffer.calculate_and_store_advantages(value_function)

# Sample a batch from the buffer for testing
sample = buffer.sample(batch_size=10)

# Unpack the sample
if buffer.use_gae:
    states, actions, rewards, next_states, dones, advantages = sample
    print("Sampled States:", states)
    print("Sampled Actions:", actions)
    print("Sampled Rewards:", rewards)
    print("Sampled Next States:", next_states)
    print("Sampled Dones:", dones)
    print("Sampled Advantages:", advantages)
else:
    states, actions, rewards, next_states, dones = sample
    print("Sampled States:", states)
    print("Sampled Actions:", actions)
    print("Sampled Rewards:", rewards)
    print("Sampled Next States:", next_states)
    print("Sampled Dones:", dones)

In [ ]:
T.set_printoptions(threshold=10000)
print(values)

In [ ]:
values.shape

In [ ]:
buffer.rewards[:buffer.counter]

In [ ]:
buffer.advantages[:buffer.counter]

In [ ]:
0.969931+0.99*0.95*0.9931

In [ ]:
1.029531+0.99*0.95*1.9039